# AWS 리소스 설정

멀티모달 RAG 챗봇에 필요한 AWS 리소스들을 생성하고 설정합니다.

## 📋 생성할 리소스들
1. S3 버킷 (문서 저장용)
2. OpenSearch Serverless 컬렉션
3. Bedrock Knowledge Base
4. IAM 역할 및 정책

**⚠️ 주의**: 이 과정에서 AWS 비용이 발생할 수 있습니다.

## 1. 환경 설정 및 확인

In [ ]:
# 가상환경 생성
!python3 -m venv venv
!venv/bin/pip install --upgrade pip

In [ ]:
#requirements.txt 파일 설치
!pip3 install --upgrade pip
!pip3 install -r requirements.txt

In [ ]:
import os
# AWS 설정
os.environ['AWS_REGION'] = 'us-west-2'  # Example value
os.environ['AWS_PROFILE'] = 'default'  # Example value

# Bedrock 설정
os.environ['BEDROCK_KNOWLEDGE_BASE_ID'] = 'your-knowledge-base-id'
os.environ['BEDROCK_MODEL_ID'] = 'anthropic.claude-3-7-sonnet-20240620-v1:0'
os.environ['BEDROCK_EMBEDDING_MODEL_ID'] = 'amazon.titan-embed-text-v2:0'
os.environ['BEDROCK_RERANK_MODEL_ID'] = 'cohere.rerank-v3-5:0'

# S3 설정
os.environ['S3_BUCKET_NAME'] = 'your-s3-bucket-name'
os.environ['S3_PREFIX'] = 'documents/'

# 애플리케이션 설정
os.environ['MAX_FILE_SIZE_MB'] = '10'
os.environ['SUPPORTED_FILE_TYPES'] = 'pdf,txt,md,jpg,jpeg,png'

In [ ]:
import boto3
import json
import time
from datetime import datetime
from botocore.exceptions import ClientError

# AWS 클라이언트 초기화
region = 'us-west-2'
session = boto3.Session()

# 계정 ID 가져오기
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()['Account']

s3_client = boto3.client('s3', region_name=region)
bedrock_client = boto3.client('bedrock', region_name=region)
bedrock_agent_client = boto3.client('bedrock-agent', region_name=region)
iam_client = boto3.client('iam', region_name=region)
opensearch_client = boto3.client('opensearchserverless', region_name=region)

print(f"✅ AWS 클라이언트 초기화 완료 (Region: {region})")

## 2. S3 버킷 생성

In [ ]:
# S3 버킷 이름 설정 (고유해야 함)
import uuid
bucket_suffix = str(uuid.uuid4())[:8]

# 1. 데이터 소스용 버킷 (문서 저장)
data_bucket_name = f"my-rag-chatbot-{bucket_suffix}"

# 2. 멀티모달 저장소용 버킷 (supplementalDataStorageConfiguration)
multimodal_bucket_name = f"multimodal-kb-storage-{bucket_suffix}"

print(f"생성할 데이터 소스 S3 버킷: {data_bucket_name}")
print(f"생성할 멀티모달 저장소 S3 버킷: {multimodal_bucket_name}")

In [ ]:
def create_s3_buckets(data_bucket_name, multimodal_bucket_name, region='us-west-2'):
    """데이터 소스용과 멀티모달 저장소용 S3 버킷 생성"""
    
    def create_single_bucket(bucket_name, bucket_type):
        """단일 S3 버킷 생성 헬퍼 함수"""
        try:
            # us-west-2 리전에서는 LocationConstraint 필요
            s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
            
            # 버킷 버전 관리 활성화
            s3_client.put_bucket_versioning(
                Bucket=bucket_name,
                VersioningConfiguration={'Status': 'Enabled'}
            )
            
            print(f"✅ {bucket_type} S3 버킷 생성 완료: {bucket_name}")
            return True
            
        except ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code == 'BucketAlreadyExists':
                print(f"❌ 버킷 이름이 이미 존재합니다: {bucket_name}")
            elif error_code == 'BucketAlreadyOwnedByYou':
                print(f"⚠️  {bucket_type} 버킷이 이미 존재합니다: {bucket_name}")
                return True
            else:
                print(f"❌ {bucket_type} S3 버킷 생성 오류: {e}")
            return False
    
    # 1. 데이터 소스용 버킷 생성
    data_success = create_single_bucket(data_bucket_name, "데이터 소스용")
    
    # 2. 멀티모달 저장소용 버킷 생성
    multimodal_success = create_single_bucket(multimodal_bucket_name, "멀티모달 저장소용")
    
    return data_success, multimodal_success

# S3 버킷들 생성 실행
data_bucket_success, multimodal_bucket_success = create_s3_buckets(data_bucket_name, multimodal_bucket_name, region)

if data_bucket_success and multimodal_bucket_success:
    print(f"\n🎉 모든 S3 버킷 생성 완료!")
    print(f"📁 데이터 소스 버킷: {data_bucket_name}")
    print(f"🗂️ 멀티모달 저장소 버킷: {multimodal_bucket_name}")
else:
    print("⚠️ 일부 버킷 생성에 실패했습니다.")

## 3. IAM 역할 생성 (Bedrock Knowledge Base용)

In [ ]:
# IAM 역할 이름
kb_role_name = f"BedrockKnowledgeBaseRole-{bucket_suffix}"

# 신뢰 정책 (Bedrock이 이 역할을 사용할 수 있도록)
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

print(f"생성할 IAM 역할 이름: {kb_role_name}")

In [ ]:
def create_iam_role_for_kb(role_name, data_bucket_name, multimodal_bucket_name, collection_arn):
    """Knowledge Base용 IAM 역할 생성 (2개 S3 버킷 지원)"""
    try:
        # IAM 역할 생성
        response = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description='Role for Bedrock Knowledge Base to access S3 and OpenSearch'
        )
        
        role_arn = response['Role']['Arn']
        print(f"✅ IAM 역할 생성 완료: {role_name}")
        print(f"Role ARN: {role_arn}")
        
        # 1. S3 접근 정책
        s3_policy = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": [
                        "s3:GetObject",
                        "s3:PutObject",
                        "s3:DeleteObject",
                        "s3:ListBucket"
                    ],
                    "Resource": [
                        f"arn:aws:s3:::{data_bucket_name}",
                        f"arn:aws:s3:::{data_bucket_name}/*",
                        f"arn:aws:s3:::{multimodal_bucket_name}",
                        f"arn:aws:s3:::{multimodal_bucket_name}/*"
                    ]
                }
            ]
        }
        
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName='S3AccessPolicy',
            PolicyDocument=json.dumps(s3_policy)
        )
        print("✅ S3 접근 정책 연결 완료")

        # 2. OpenSearch 접근 정책
        opensearch_policy = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": [
                        "aoss:APIAccessAll"
                    ],
                    "Resource": [
                        "arn:aws:aoss:us-west-2:*:collection/*"  
                    ]
                }
            ]
        }
        
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName='OpenSearchServerlessPolicy',
            PolicyDocument=json.dumps(opensearch_policy)
        )
        print("✅ OpenSearch 접근 정책 연결 완료")
        
        # 3. Foundation Model 정책 (새로 추가)
        fm_policy = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": [
                        "bedrock:InvokeModel"
                    ],
                    "Resource": [
                        f"arn:aws:bedrock:{region}::foundation-model/amazon.titan-embed-text-v2:0"
                    ]
                }
            ]
        }
        
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName='BedrockFoundationModelPolicy',
            PolicyDocument=json.dumps(fm_policy)
        )
        print("✅ Foundation Model 정책 연결 완료")

        # 4. Rerank 정책 추가
        rerank_policy = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": ["bedrock:Rerank"],
                    "Resource": ["*"]
                }
            ]
        }

        iam_client.put_role_policy(
            RoleName=role_name,  # 수정됨
            PolicyName='BedrockRerankPolicy',
            PolicyDocument=json.dumps(rerank_policy)
        )
        print("✅ Rerank 정책 연결 완료")

        # 5. Inference Profile 정책 추가
        inference_policy = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": [
                        "bedrock:GetInferenceProfile",
                        "bedrock:InvokeModel"
                    ],
                    "Resource": [
                    f"arn:aws:bedrock:{region}:{account_id}:inference-profile/us.anthropic.claude-3-7-sonnet-20250219-v1:0",
                    f"arn:aws:bedrock:{region}:{account_id}:application-inference-profile/us.anthropic.claude-3-7-sonnet-20250219-v1:0",
                    f"arn:aws:bedrock:*::foundation-model/anthropic.claude-3-7-sonnet-20250219-v1:0"
                    ]
                }
            ]
        }

        iam_client.put_role_policy(
            RoleName=role_name,  # 수정됨
            PolicyName='BedrockInferenceProfilePolicy',
            PolicyDocument=json.dumps(inference_policy)
        )
        print("✅ Inference Profile 정책 추가 완료")

        return role_arn

    except ClientError as e:
        if e.response['Error']['Code'] == 'EntityAlreadyExists':
            print(f"⚠️  IAM 역할이 이미 존재합니다: {role_name}")
            response = iam_client.get_role(RoleName=role_name)
            return response['Role']['Arn']
        else:
            print(f"❌ IAM 역할 생성 오류: {e}")
            return None

# IAM 역할 생성 실행 (collection_arn 추가)
kb_role_arn = create_iam_role_for_kb(kb_role_name, data_bucket_name, multimodal_bucket_name, collection_arn)
print(f"\nKnowledge Base Role ARN: {kb_role_arn}")

## 4. OpenSearch Serverless 컬렉션과 인덱스 생성

In [ ]:
# OpenSearch Serverless 컬렉션 이름
collection_name = f"my-rag-{bucket_suffix}"

print(f"생성할 OpenSearch Serverless 컬렉션: {collection_name}")

In [ ]:
def create_opensearch_collection_and_index(collection_name):
    """OpenSearch Serverless 컬렉션과 벡터 인덱스 생성"""
    
    def create_vector_index(endpoint):
        """벡터 인덱스 생성 헬퍼 함수"""
        from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
        
        try:
            # AWS 인증 설정
            credentials = boto3.Session().get_credentials()
            auth = AWSV4SignerAuth(credentials, region, 'aoss')
            
            # OpenSearch 클라이언트 생성
            host = endpoint.replace('https://', '')
            client = OpenSearch(
                hosts=[{'host': host, 'port': 443}],
                http_auth=auth,
                use_ssl=True,
                verify_certs=True,
                connection_class=RequestsHttpConnection,
                pool_maxsize=20
            )
            
            # 벡터 인덱스 설정
            index_body = {
                'settings': {
                    'index': {
                        'knn': True,
                        'knn.algo_param.ef_search': 512
                    }
                },
                'mappings': {
                    'properties': {
                        'bedrock-knowledge-base-default-vector': {
                            'type': 'knn_vector',
                            'dimension': 1024,
                            'method': {
                                'name': 'hnsw',
                                'space_type': 'l2',
                                'engine': 'faiss',
                                'parameters': {
                                    'ef_construction': 512,
                                    'm': 16
                                }
                            }
                        },
                        'AMAZON_BEDROCK_TEXT': {
                            'type': 'text',
                            "fields": {
                                "keyword": {
                                  "type": "keyword"
                                }
                            }
                        },
                        'AMAZON_BEDROCK_METADATA': {
                            'type': 'text',
                            'index': False
                        },
                        'id': {
                            'type': 'text',
                            'fields': {
                                'keyword': {
                                    'type': 'keyword',
                                    'ignore_above': 256
                                }
                            }
                        },
                        'x-amz-bedrock-kb-data-source-id': {
                            'type': 'text',
                            'fields': {
                                'keyword': {
                                    'type': 'keyword',
                                    'ignore_above': 256
                                }
                            }
                        },
                        'x-amz-bedrock-kb-document-page-number': {
                            'type': 'long'
                        },
                        'x-amz-bedrock-kb-source-uri': {
                            'type': 'text',
                            'fields': {
                                'keyword': {
                                    'type': 'keyword',
                                    'ignore_above': 256
                                }
                            }
                        }
                    }
                }
            }
            
            # 인덱스 생성
            response = client.indices.create(
                index='bedrock-knowledge-base-default-index',
                body=index_body
            )
            
            if response.get('acknowledged', False):
                return True
            else:
                print(f'인덱스 생성 실패: {response}')
                return False
                
        except Exception as e:
            print(f'인덱스 생성 중 오류: {e}')
            return False
    
    try:
        # 보안 정책 생성 (암호화)
        encryption_policy = {
            "Rules": [
                {
                    "ResourceType": "collection",
                    "Resource": [f"collection/{collection_name}"]
                }
            ],
            "AWSOwnedKey": True
        }
        
        opensearch_client.create_security_policy(
            name=f"{collection_name}",
            type='encryption',
            policy=json.dumps(encryption_policy)
        )
        
        print("✅ 암호화 정책 생성 완료")
        
        # 네트워크 정책 생성 (퍼블릭 액세스)
        network_policy = [
            {
                "Rules": [
                    {
                        "ResourceType": "collection",
                        "Resource": [f"collection/{collection_name}"]
                    },
                    {
                        "ResourceType": "dashboard",
                        "Resource": [f"collection/{collection_name}"]
                    }
                ],
                "AllowFromPublic": True
            }
        ]
        
        opensearch_client.create_security_policy(
            name=f"{collection_name}-network",
            type='network',
            policy=json.dumps(network_policy)
        )
        
        print("✅ 네트워크 정책 생성 완료")
        
        # 데이터 접근 정책 생성
        data_access_policy = [
            {
                "Rules": [
                    {
                        "ResourceType": "collection",
                        "Resource": [f"collection/{collection_name}"],
                        "Permission": [
                            "aoss:CreateCollectionItems",
                            "aoss:DeleteCollectionItems",
                            "aoss:UpdateCollectionItems",
                            "aoss:DescribeCollectionItems"
                        ]
                    },
                    {
                        "ResourceType": "index",
                        "Resource": [f"index/{collection_name}/*"],
                        "Permission": [
                            "aoss:CreateIndex",
                            "aoss:DeleteIndex",
                            "aoss:UpdateIndex",
                            "aoss:DescribeIndex",
                            "aoss:ReadDocument",
                            "aoss:WriteDocument"
                        ]
                    }
                ],
                "Principal": [kb_role_arn]
            }
        ]
        
        opensearch_client.create_access_policy(
            name=f"{collection_name}-access",
            type='data',
            policy=json.dumps(data_access_policy)
        )
        print("✅ 데이터 접근 정책 생성 완료")

        # 컬렉션 생성
        response = opensearch_client.create_collection(
            name=collection_name,
            type='VECTORSEARCH',
            description='Vector collection for RAG chatbot',
            standbyReplicas='DISABLED'
        )
        
        collection_id = response['createCollectionDetail']['id']
        collection_arn = response['createCollectionDetail']['arn']
        
        print(f"✅ OpenSearch Serverless 컬렉션 생성 시작")
        print(f"Collection ID: {collection_id}")
        print(f"Collection ARN: {collection_arn}")
        
        # 컬렉션 생성 완료 대기
        print("⏳ 컬렉션 생성 완료 대기 중...")
        
        while True:
            response = opensearch_client.list_collections(
                collectionFilters={'name': collection_name}
            )
            
            if response['collectionSummaries']:
                status = response['collectionSummaries'][0]['status']
                print(f"현재 상태: {status}")
                
                if status == 'ACTIVE':
                    detail_response = opensearch_client.batch_get_collection(
                        names=[collection_name]
                    )
                    
                    if detail_response['collectionDetails']:
                        collection_detail = detail_response['collectionDetails'][0]
                        endpoint = collection_detail['collectionEndpoint']
                        
                        print(f"✅ 컬렉션 생성 완료!")
                        print(f"Endpoint: {endpoint}")
                        
                        # 벡터 인덱스 생성
                        print("⏳ 벡터 인덱스 생성 중...")
                        index_created = create_vector_index(endpoint)
                        
                        if index_created:
                            print("✅ 벡터 인덱스 생성 완료!")
                        else:
                            print("⚠️ 벡터 인덱스 생성 실패 (Knowledge Base 생성 시 자동 생성됨)")
                        
                        return collection_id, collection_arn, endpoint
                    else:
                        print("❌ 컬렉션 세부 정보를 가져올 수 없습니다")
                        return collection_id, collection_arn, None
                        
                elif status == 'FAILED':
                    print("❌ 컬렉션 생성 실패")
                    return None, None, None
            
            time.sleep(30)  # 30초 대기
            
    except ClientError as e:
        print(f"❌ OpenSearch Serverless 컬렉션 생성 오류: {e}")
        return None, None, None

# OpenSearch Serverless 컬렉션과 인덱스 생성 실행
collection_id, collection_arn, collection_endpoint = create_opensearch_collection_and_index(collection_name)

## 5. Bedrock Knowledge Base 생성

In [ ]:
# Knowledge Base 이름
kb_name = f"my-rag-kb-{bucket_suffix}"

print(f"생성할 Knowledge Base 이름: {kb_name}")

In [ ]:
def create_knowledge_base(kb_name, data_bucket_name, multimodal_bucket_name, collection_arn, role_arn):
    """Bedrock Knowledge Base 생성 (컬렉션과 인덱스 기반)"""
    try:
        response = bedrock_agent_client.create_knowledge_base(
            name=kb_name,
            description='Knowledge Base for RAG chatbot',
            roleArn=role_arn, 
            knowledgeBaseConfiguration={
                'type': 'VECTOR',
                'vectorKnowledgeBaseConfiguration': {
                    'embeddingModelArn': f'arn:aws:bedrock:{region}::foundation-model/amazon.titan-embed-text-v2:0',
                    'embeddingModelConfiguration': {
                        'bedrockEmbeddingModelConfiguration': {
                            'dimensions': 1024,
                            'embeddingDataType': 'FLOAT32'
                        }
                    },
                    'supplementalDataStorageConfiguration': {
                        'storageLocations': [
                            {
                                's3Location': {
                                    'uri': f's3://{multimodal_bucket_name}'
                                },
                                'type': 'S3'
                            }
                        ]
                    }
                }
            },
            storageConfiguration={
                'type': 'OPENSEARCH_SERVERLESS',
                'opensearchServerlessConfiguration': {
                    'collectionArn': collection_arn,
                    'vectorIndexName': 'bedrock-knowledge-base-default-index',
                    'fieldMapping': {
                        'vectorField': 'bedrock-knowledge-base-default-vector',
                        'textField': 'AMAZON_BEDROCK_TEXT',
                        'metadataField': 'AMAZON_BEDROCK_METADATA'
                    }
                }
            }
        )
        
        kb_id = response['knowledgeBase']['knowledgeBaseId']
        kb_arn = response['knowledgeBase']['knowledgeBaseArn']
        
        print(f"✅ Knowledge Base 생성 완료!")
        print(f"Knowledge Base ID: {kb_id}")
        print(f"Knowledge Base ARN: {kb_arn}")
        
        return kb_id, kb_arn
        
    except ClientError as e:
        print(f"❌ Knowledge Base 생성 오류: {e}")
        # 이미 존재하는 경우 기존 KB 정보 조회
        if 'already exists' in str(e):
            print("🔍 기존 Knowledge Base 정보를 조회합니다...")
            try:
                kb_list = bedrock_agent_client.list_knowledge_bases()
                for kb in kb_list['knowledgeBaseSummaries']:
                    if kb['name'] == kb_name:
                        existing_kb_id = kb['knowledgeBaseId']
                        kb_detail = bedrock_agent_client.get_knowledge_base(knowledgeBaseId=existing_kb_id)
                        existing_kb_arn = kb_detail['knowledgeBase']['knowledgeBaseArn']
                        print(f"✅ 기존 Knowledge Base 사용: {existing_kb_id}")
                        return existing_kb_id, existing_kb_arn
            except Exception as lookup_error:
                print(f"❌ 기존 Knowledge Base 조회 오류: {lookup_error}")
        return None, None

# Knowledge Base 생성 (컬렉션과 인덱스가 준비된 경우에만)
if collection_arn and collection_endpoint:
    print("\n🚀 Knowledge Base 생성 시작...")
    kb_id, kb_arn = create_knowledge_base(kb_name, data_bucket_name, multimodal_bucket_name, collection_arn, kb_role_arn)
else:
    print("❌ OpenSearch 컬렉션과 인덱스가 준비되지 않아 Knowledge Base를 생성할 수 없습니다.")
    kb_id, kb_arn = None, None

print(f"✅ Knowledge Base ID: {kb_id}")  

## 6. 데이터 소스 생성 (S3 연결)

In [ ]:
def create_data_source(kb_id, data_bucket_name):
    """Knowledge Base에 S3 데이터 소스 연결"""
    try:
        response = bedrock_agent_client.create_data_source(
            knowledgeBaseId=kb_id,
            name=f"multimodal-data-source-{bucket_suffix}",
            description='Multimodal S3 data source with Claude parsing',
            dataSourceConfiguration={
                'type': 'S3',
                's3Configuration': {
                    'bucketArn': f'arn:aws:s3:::{data_bucket_name}',
                    'inclusionPrefixes': ['documents/']
                }
            },
            vectorIngestionConfiguration={
                'chunkingConfiguration': {
                    'chunkingStrategy': 'HIERARCHICAL',
                    'hierarchicalChunkingConfiguration': {
                        'levelConfigurations': [
                            {
                                'maxTokens': 1500
                            },
                            {
                                'maxTokens': 300
                            }
                        ],
                        'overlapTokens': 60
                    }
                },
                'parsingConfiguration': {
                    'parsingStrategy': 'BEDROCK_FOUNDATION_MODEL',
                    'bedrockFoundationModelConfiguration': {
                        'modelArn': f'arn:aws:bedrock:{region}:{account_id}:inference-profile/us.anthropic.claude-3-7-sonnet-20250219-v1:0',
                        'parsingModality': 'MULTIMODAL'
                    }
                }
            }
        )
        
        data_source_id = response['dataSource']['dataSourceId']
        
        print(f"✅ 데이터 소스 생성 완료!")
        print(f"Data Source ID: {data_source_id}")
        
        return data_source_id
        
    except ClientError as e:
        print(f"❌ 데이터 소스 생성 오류: {e}")
        return None

# 데이터 소스 생성 실행
if kb_id:
    data_source_id = create_data_source(kb_id, data_bucket_name)
else:
    print("❌ Knowledge Base가 준비되지 않아 데이터 소스를 생성할 수 없습니다.")
    data_source_id = None

## 7. 생성된 리소스 정보 요약

In [ ]:
print("🎉 AWS 멀티모달 RAG 리소스 생성 완료!")
print("=" * 60)

print("📦 S3 버킷들:")
print(f"   - 데이터 소스: {data_bucket_name}")
print(f"   - 멀티모달 저장소: {multimodal_bucket_name}")

print(f"\n🔍 OpenSearch Serverless:")
print(f"   - 컬렉션: {collection_name}")
if collection_endpoint:
    print(f"   - Endpoint: {collection_endpoint}")
print(f"   - 벡터 인덱스: bedrock-knowledge-base-default-index")
print(f"   - 암호화 정책: {collection_name}")
print(f"   - 네트워크 정책: {collection_name}-network")
print(f"   - 데이터 접근 정책: {collection_name}-access")

print(f"\n🧠 Bedrock Knowledge Base:")
print(f"   - 이름: {kb_name}")
if kb_id:
    print(f"   - ID: {kb_id}")
print(f"   - 임베딩 모델: Titan Text v2 (1024차원, FLOAT32)")
print(f"   - 멀티모달 지원: 활성화")

if data_source_id:
    print(f"\n📁 데이터 소스:")
    print(f"   - ID: {data_source_id}")
    print(f"   - 파싱: Claude 3.7 Sonnet (멀티모달)")
    print(f"   - 청킹: 계층형 (1500→300 토큰, 60 오버랩)")

print(f"\n👤 IAM 역할: {kb_role_name}")
print(f"   - S3 접근: 2개 버킷 (읽기/쓰기)")
print(f"   - OpenSearch 접근: 전체 권한")
print(f"   - Bedrock 모델: Titan, Claude, Rerank")

print("\n📝 환경 변수 업데이트 정보:")
print(f"S3_BUCKET_NAME={data_bucket_name}")
print(f"MULTIMODAL_BUCKET_NAME={multimodal_bucket_name}")
if kb_id:
    print(f"BEDROCK_KNOWLEDGE_BASE_ID={kb_id}")
if collection_name:
    print(f"OPENSEARCH_COLLECTION_NAME={collection_name}")
print(f"BEDROCK_MODEL_ID=anthropic.claude-3-7-sonnet-20250219-v1:0")
print(f"BEDROCK_EMBEDDING_MODEL_ID=amazon.titan-embed-text-v2:0")

## 8. 환경 변수 자동 업데이트

In [ ]:
import os

# 환경 변수 업데이트
if data_bucket_name:
    os.environ['S3_BUCKET_NAME'] = data_bucket_name
    print(f"✅ S3_BUCKET_NAME 업데이트: {data_bucket_name}")

if multimodal_bucket_name:
    os.environ['MULTIMODAL_BUCKET_NAME'] = multimodal_bucket_name
    print(f"✅ MULTIMODAL_BUCKET_NAME 업데이트: {multimodal_bucket_name}")

if kb_id:
    os.environ['BEDROCK_KNOWLEDGE_BASE_ID'] = kb_id
    print(f"✅ BEDROCK_KNOWLEDGE_BASE_ID 업데이트: {kb_id}")

if collection_name:
    os.environ['OPENSEARCH_COLLECTION_NAME'] = collection_name
    print(f"✅ OPENSEARCH_COLLECTION_NAME 업데이트: {collection_name}")

if data_source_id:
    os.environ['BEDROCK_DATA_SOURCE_ID'] = data_source_id
    print(f"✅ BEDROCK_DATA_SOURCE_ID 업데이트: {data_source_id}")

# Bedrock 모델 설정
os.environ['BEDROCK_MODEL_ID'] = 'anthropic.claude-3-7-sonnet-20250219-v1:0'
os.environ['BEDROCK_EMBEDDING_MODEL_ID'] = 'amazon.titan-embed-text-v2:0'
os.environ['BEDROCK_RERANK_MODEL_ID'] = 'cohere.rerank-v3-5:0'
print(f"✅ BEDROCK_MODEL_ID 업데이트: Claude 3.7 Sonnet")
print(f"✅ BEDROCK_EMBEDDING_MODEL_ID 업데이트: Titan Text v2")
print(f"✅ BEDROCK_RERANK_MODEL_ID 업데이트: Cohere Rerank v3")

# 애플리케이션 설정
os.environ['AWS_REGION'] = region
os.environ['AWS_ACCOUNT_ID'] = account_id
os.environ['MAX_FILE_SIZE_MB'] = '10'
os.environ['SUPPORTED_FILE_TYPES'] = 'pdf,txt,md,jpg,jpeg,png'
print(f"✅ AWS_REGION 업데이트: {region}")
print(f"✅ AWS_ACCOUNT_ID 업데이트: {account_id}")
print(f"✅ 애플리케이션 설정 업데이트 완료")

# .env 파일 생성
env_content = f'''# AWS 설정
AWS_REGION={region}
AWS_PROFILE=default
AWS_ACCOUNT_ID={account_id}

# S3 설정
S3_BUCKET_NAME={data_bucket_name if data_bucket_name else 'your-data-bucket'}
MULTIMODAL_BUCKET_NAME={multimodal_bucket_name if multimodal_bucket_name else 'your-multimodal-bucket'}
S3_PREFIX=documents/

# Bedrock 설정
BEDROCK_KNOWLEDGE_BASE_ID={kb_id if kb_id else 'your-knowledge-base-id'}
BEDROCK_DATA_SOURCE_ID={data_source_id if data_source_id else 'your-data-source-id'}
BEDROCK_MODEL_ID=anthropic.claude-3-7-sonnet-20250219-v1:0
BEDROCK_EMBEDDING_MODEL_ID=amazon.titan-embed-text-v2:0
BEDROCK_RERANK_MODEL_ID=cohere.rerank-v3-5:0

# OpenSearch 설정
OPENSEARCH_COLLECTION_NAME={collection_name if collection_name else 'your-collection-name'}

# 애플리케이션 설정
MAX_FILE_SIZE_MB=10
SUPPORTED_FILE_TYPES=pdf,txt,md,jpg,jpeg,png
'''

# .env 파일 저장
try:
    with open('.env', 'w', encoding='utf-8') as f:
        f.write(env_content)
    print(f"\n📄 .env 파일 생성 완료!")
    print(f"   위치: {os.path.abspath('.env')}")
    print(f"   내용: 모든 환경 변수가 저장되었습니다")
except Exception as e:
    print(f"\n❌ .env 파일 생성 실패: {e}")

print("\n🎯 다음 단계:")
print("1. S3 버킷들에 문서 업로드 (텍스트, 이미지, PDF)")
print("2. Knowledge Base 데이터 소스 동기화")
print("3. 멀티모달 RAG 챗봇 코드 구현 시작")
print("4. Streamlit UI 개발")
print("\n💡 팁: 생성된 .env 파일을 확인하고 필요시 수정하세요!")
print("⚠️  주의: .env 파일을 Git에 커밋하지 마세요 (.gitignore에 추가됨)")